# Basic GTM Training for S-curve

## Importing libraries

In [1]:
import os
import numpy as np
import torch
from sklearn.datasets import make_s_curve

# GTM and utils
from gtmkit.gtm import GTM
from gtmkit.utils.molecules import calculate_latent_coords
from gtmkit.utils.regression import get_reg_density_matrix, reg_density_to_table
from gtmkit.utils.density import density_to_table

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401  # needed for 3D subplot

# Needed for plotply plots
import plotly.graph_objects as go
import ipywidgets as widgets

import altair as alt
from gtmkit.plots.altair_landscapes import (
    altair_points_chart,
    altair_discrete_regression_landscape,
    altair_discrete_density_landscape
)
from gtmkit.plots.plotly_landscapes import plotly_smooth_density_landscape, plotly_smooth_regression_landscape


## Creating an S-curve dataset

In [70]:
# -----------------------
# 1) Data: S-curve in 3D + extra noise dim
# -----------------------
rng = np.random.RandomState(0)
n_samples = 5000

# S-curve: (n_samples, 3) and per-sample "color" parameter along the curve
s_curve_3d, s_curve_color = make_s_curve(n_samples, random_state=rng)

# Torch tensor (double precision often helps EM stability)
device = "cuda" if torch.cuda.is_available() else "cpu"
X_t = torch.tensor(s_curve_3d, dtype=torch.float64, device=device)

## Building the GTM

In [71]:
# -----------------------
# 2) Fit GTM
# -----------------------
gtm = GTM(
    num_nodes=16**2,             # 15 x 15 grid
    num_basis_functions=2**2,   # 10 x 10 RBF centers
    basis_width=1,
    reg_coeff=0.1,
    device=device,
    standardize=False,         # set True if your real data needs z-scoring
    pca_scale=True,
    pca_engine="sklearn",
    max_iter=200,
)
gtm.fit(X_t)

  0%|          | 0/200 [00:00<?, ?it/s]

## Plotting the results with matplotlib

Latent coordinates shape: (5000, 2)


In [72]:
Z = gtm.transform(X_t)  # [n_samples, 2]
print(f"Latent coordinates shape: {tuple(Z.shape)}")

# -----------------------
# 3) Immediate visualization (matplotlib)
# 1x2 subplots: original 3D vs latent 2D
# -----------------------
Z_np = Z.detach().to("cpu").numpy() if hasattr(Z, "detach") else np.asarray(Z)
X3d_np = np.asarray(s_curve_3d)
color = np.asarray(s_curve_color)

fig = plt.figure(figsize=(12, 5), constrained_layout=True)

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
p1 = ax1.scatter(
    X3d_np[:, 0], X3d_np[:, 1], X3d_np[:, 2],
    c=color, cmap='viridis', s=3, alpha=0.8
)
ax1.set_title('Original S-curve (3D)')
ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')
fig.colorbar(p1, ax=ax1, fraction=0.046, pad=0.04, label='Curve parameter')

ax2 = fig.add_subplot(1, 2, 2)
p2 = ax2.scatter(
    Z_np[:, 0], Z_np[:, 1],
    c=color, cmap='viridis', s=3, alpha=0.8
)
ax2.set_title('GTM latent space (2D)')
ax2.set_xlabel('Latent 1'); ax2.set_ylabel('Latent 2')
fig.colorbar(p2, ax=ax2, fraction=0.046, pad=0.04, label='Curve parameter')

plt.show()

## Interactive plotting with Altair: scatter plot and regression density landscape

In [73]:
# -----------------------
# 4) Responsibilities for landscapes
# -----------------------
responsibilities, log_likelihoods = gtm.project(X_t)

# Convert responsibilities to NumPy on CPU and fix orientation to (n_samples, n_nodes)
R = responsibilities.cpu().numpy()

n_nodes = R.shape[1]
grid_size = int(np.sqrt(n_nodes))
assert grid_size * grid_size == n_nodes, "num_nodes must be a perfect square."

# -----------------------
# 5) Regression landscape (no density chart)
#    Left: latent points (Altair)
#    Right: regression landscape (Altair)
# -----------------------
# Use the intrinsic curve parameter as the regression target
regression_values = s_curve_color

# Node densities and per-node regression values
density, reg_density = get_reg_density_matrix(R, regression_values)

dens_table = density_to_table(density)

# Tidy table for the regression landscape
reg_table = reg_density_to_table(
    density,
    reg_density,
    node_threshold=0.10  # ignore very sparse nodes for cleaner viz
)

coords = calculate_latent_coords(
    R,
    correction=True,
    return_node=True
)
coords["color"] = regression_values

### Points chart

In [74]:
# Latent coordinates for samples (for the points panel)

# Points (left) — use same viridis scheme for consistency with matplotlib
points_chart = altair_points_chart(
    coords.sample(min(5000, len(coords)), random_state=0),
    num_nodes=n_nodes,
    points_size=100,
    coloring_scheme='viridis',
    coloring_column='color',
    legend=None,
    tooltip_columns={"node_index": "Node: ", "color": "Color: "}
)

# Density is used as base layer of the points chart
chart_density = altair_discrete_density_landscape(dens_table)

### Regression landscape

In [75]:
# Regression landscape (right)
reg_chart = altair_discrete_regression_landscape(
    reg_table,
    title="GTM Regression Landscape",
    colorset='viridis',
    regression_label="Color"
)

### Combining landscapes and saving them

In [76]:
# --- make the two panels the same size and aligned ---
SIZE = 400  # pick any; same for both

combined = alt.hconcat(
    (chart_density + points_chart).properties(width=SIZE, height=SIZE),
    reg_chart.properties(width=SIZE, height=SIZE)).properties(title="Latent points (colored by target) — GTM Regression Landscape")

# Try displaying inline; always save to HTML as well
out_dir = "plots"
os.makedirs(out_dir, exist_ok=True)
html_path = os.path.join(out_dir, "gtm_scurve_regression.html")
combined.save(html_path)
print(f"Saved Altair visualization to {html_path}")

# Optionally display in notebooks that support Altair rendering:
combined

Saved Altair visualization to plots/gtm_scurve_regression.html


alt.HConcatChart(...)

In [77]:
# One could also showcase the landscapes using the smooth plotly implementation

plotly_density = plotly_smooth_density_landscape(dens_table, width=600, height=600)
plotly_reg     = plotly_smooth_regression_landscape(reg_table, colorset='viridis', legend_label='Color', regression_label='Color', width=600, height=600)

fig1_widget = go.FigureWidget(plotly_density)
fig2_widget = go.FigureWidget(plotly_reg)

widgets_box = widgets.HBox([fig1_widget, fig2_widget])
widgets_box

    'data': [{'colorbar': {'title': {'text': 'Density'}},
              'colorsc…

---